
# Detection / Segmentation’da “Saçmalama” (Kararsız Eğitim) Nasıl Önlenir?
## Normalization + Stabilizasyon Paketi (Tek GPU / Küçük Batch Odaklı)

Bu notebook’un amacı: **tek GPU** ve **küçük batch (1–4–8)** ile detection/segmentation eğitirken
yaşanan “loss zıplaması / mAP-IOU oturmaması / training patlaması” gibi problemleri **mühendislik mantığıyla** açıklamak
ve **pratik çözümleri** kod şablonlarıyla vermek.

> Kısaca: “Saçmalama” çoğunlukla **istatistik gürültüsü + ölçek dalgalanması + agresif optimizasyon** birleşimidir.



## 1) “Saçmalama” dediğimiz şey tam olarak ne?

Detection/segmentation eğitiminde tipik belirtiler:
- Loss’un aşırı dalgalanması (step’ten step’e büyük zıplama)
- mAP / IoU’nun uzun süre oturmaması veya “geri gitmesi”
- Grad norm’un patlaması, NaN/Inf
- Aynı seed ile bile çok farklı sonuçlar (yüksek varyans)

Bu belirtiler çoğu zaman “model kötü” değil; **eğitim dinamikleri kararsız** demek.



## 2) Kök nedenler (en sık görülen 3 kaynak)

### 2.1. Küçük batch istatistiği (BN problemi)
- Detection/segmentation’da input büyük → VRAM yetmez → batch küçük
- BatchNorm train’de mean/var’ı batch’ten alır
- Batch küçük olunca mean/var gürültülü olur → aktivasyon ölçeği zıplar

### 2.2. Feature map ölçeği (weight/activation scale drift)
- Derin conv bloklarında aktivasyon ölçeği katman katman kayabilir
- Özellikle residual + ağır augment + mixed precision kombinasyonunda dalgalanma artar

### 2.3. Optimizasyon agresifliği (LR, schedule, warmup)
- Küçük batch + yüksek LR = çok sık patlama
- Warmup yoksa ilk epoch’larda ölçek oturmadan parametreler sert güncellenir



## 3) “Sağlıklı kontrol paketi” 

Tek GPU + küçük batch için en sağlam yaklaşım sırası:

1) **Normalization seçimini düzelt**
   - Default: **GroupNorm**
   - Alternatif: **Frozen BatchNorm** (pretrained backbone finetune)

2) **Ölçek stabilizasyonu ekle (opsiyonel ama güçlü)**
   - **Weight Standardization (WS)** özellikle GN ile iyi gider

3) **Optimizasyon emniyet kemerleri**
   - LR düşür, warmup ekle
   - AMP kullan (VRAM kazan → batch büyüt)
   - Gradient clipping (özellikle transformer head / ağır loss’larda)



## 4) Normalization tarafı: BN neden bozuluyor?

BatchNorm (Conv için) kabaca kanallar bazında (B,H,W) üzerinden istatistik alır.

- Train: \(\mu_B, \sigma_B\)
- Eval: running \(\mu_R, \sigma_R\)

Küçük batch’te problem:
- \(\mu_B, \sigma_B\) çok oynar
- Train’de normalize edilen dağılım step step değişir
- Eval’a geçtiğinde running stats farklı olduğundan **train-eval mismatch** artar

Bu mismatch, detection/segmentation gibi “ince ayar” gerektiren görevlerde daha acı hissedilir.



## 5) Çözüm #1: GroupNorm (GN) — Tek GPU’da default

### 5.1. GN fikri
GN batch’e bakmaz. Kanalları gruplara ayırır ve her örnek içinde normalize eder.
- Batch boyutu = 1 bile olsa stabil çalışır ✅
- Train-eval mismatch gibi bir problem yok (istatistik sample içi)

### 5.2. Nerede kullanılır?
- Detection ve segmentation pipeline’larında çok yaygın
- Özellikle küçük batch, büyük çözünürlük senaryosunda birinci tercih

### 5.3. Pratik kural
- `num_groups` genelde 32 seçilir (kanal sayısı buna bölünmeli)
- Kanal azsa 16/8 gibi seç


In [1]:
import torch
import torch.nn as nn

# GN örnek: C=64 kanallı bir blok için
gn = nn.GroupNorm(num_groups=32, num_channels=64)
x = torch.randn(2, 64, 32, 32)
y = gn(x)
y.shape

torch.Size([2, 64, 32, 32])


## 6) Çözüm #2: Frozen BatchNorm (BN freeze) — Pretrained finetune kurtarıcısı

### 6.1. Mantık
Pretrained backbone (ImageNet vb.) ile gelen BN running stats çoğu zaman “iyi”dir.
Küçük batch ile finetune yaparken bu stats’leri güncellersen:
- gürültülü batch stats running’e karışır
- modelin ölçeği bozulur

**Frozen BN**:
- BN katmanlarını eval benzeri sabit tutar (running mean/var güncellenmez)
- affine parametre (gamma/beta) istersen train edilebilir

### 6.2. Ne zaman iyi?
- Backbone pretrained
- Batch küçük
- GN’ye çevirmek istemiyorsun (pipeline karmaşıklığı)


In [2]:
# Basit BN freeze: BN modüllerini eval'a alıp stats güncellemesini kapatmak
def freeze_batchnorm_stats(model: nn.Module):
    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.eval()               # running stats kullan
            m.requires_grad_(True) # affine train et (istersen)
    return model

# Örnek kullanım:
class SmallBNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
    def forward(self, x): return self.net(x)

m = SmallBNNet()
m = freeze_batchnorm_stats(m)
m

SmallBNNet(
  (net): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
)


## 7) Çözüm #3: Weight Standardization (WS) — GN ile iyi kombin

### 7.1. WS nedir?
WS, Conv ağırlıklarını (per-out-channel) standartlaştırır:
- Her output channel için weight mean’ini çıkarır
- std’ye böler

Bu, activations’ın ölçeğini daha stabil tutar. GN ile birlikte kullanımı çok yaygındır (özellikle bazı segmentation/backbone çalışmalarında).

### 7.2. WS neyi çözmez?
- BN’in batch istatistiği sorununu tek başına çözmez
- Ama GN/LN gibi batch bağımsız normlarla stabiliteyi artırır


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WSConv2d(nn.Conv2d):
    """Weight Standardization uygulanmış Conv2d"""
    def forward(self, x):
        w = self.weight
        w_mean = w.mean(dim=(1,2,3), keepdim=True)
        w = w - w_mean
        w_std = w.flatten(1).std(dim=1, keepdim=True).view(-1,1,1,1) + 1e-5
        w = w / w_std
        return F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)

class WS_GN_Block(nn.Module):
    def __init__(self, in_ch, out_ch, groups=32):
        super().__init__()
        self.conv = WSConv2d(in_ch, out_ch, 3, padding=1, bias=False)
        self.gn = nn.GroupNorm(num_groups=min(groups, out_ch), num_channels=out_ch)
        self.act = nn.SiLU(inplace=True)
    def forward(self, x):
        return self.act(self.gn(self.conv(x)))

blk = WS_GN_Block(3, 64, groups=32)
y = blk(torch.randn(2,3,64,64))
y.shape

torch.Size([2, 64, 64, 64])


## 8) Optimizasyon emniyet kemerleri (çoğu kişinin kaçırdığı yer)

### 8.1. Learning rate (LR) küçült
Küçük batch’te gradient daha gürültülüdür → yüksek LR dalgalanmayı büyütür.

Pratik:
- Aynı modeli küçük batch ile eğitirken LR’ı düşür
- Warmup ekle

### 8.2. Warmup
İlk N iterasyonda LR’ı yavaş artır.
Bu, özellikle pretrained + yeni head senaryosunda patlamayı azaltır.

### 8.3. AMP (mixed precision)
AMP çoğu zaman:
- VRAM kazandırır → batch’i büyütebilirsin (en temiz stabilite artışı)
- throughput artırır

### 8.4. Gradient clipping (opsiyonel)
Özellikle transformer head / ağır loss kombinasyonunda:
- grad norm’un patlamasını engeller


In [4]:
# Gradient clipping örnek şablon
def train_step(model, opt, loss_fn, x, y, max_norm=1.0):
    opt.zero_grad(set_to_none=True)
    out = model(x)
    loss = loss_fn(out, y)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
    opt.step()
    return loss.item()


## 9) “Biz hangisini seçeceğiz?” Karar tablosu (tek GPU)

### A) Yeni bir detection/segmentation modeli yazıyorsak
✅ **GroupNorm** (default)  
Opsiyonel: **WS + GN**

### B) Pretrained backbone ile finetune yapıyorsak (çok yaygın)
✅ **Frozen BN** veya **BN→GN dönüşümü**  
- Eğer pipeline BN’e bağımlıysa: Frozen BN çok pratik
- Eğer daha stabil istiyorsan: GN (veya WS+GN) daha temiz

### C) Batch 1 ise
✅ GN / WS+GN  
⚠️ BN tavsiye edilmez (frozen değilse)

> BatchRenorm bu dünyada genelde “niş/ara çözüm”. GN varken çoğu zaman tercih edilmez.



## 10) Mini demo: Küçük batch’te BN vs GN istatistik farkı (sezgisel)

Amaç:
- BN’in batch küçülünce ne kadar oynaklaşabileceğini görmek
- GN’in batch’ten bağımsız olduğuna sezgi kazanmak


In [ ]:
import torch
import torch.nn as nn

torch.manual_seed(0)

bn = nn.BatchNorm2d(16).train()
gn = nn.GroupNorm(8, 16).train()

x1 = torch.randn(1, 16, 32, 32)  # batch=1
x8 = torch.randn(8, 16, 32, 32)  # batch=8

y_bn_1 = bn(x1)
y_bn_8 = bn(x8)

y_gn_1 = gn(x1)
y_gn_8 = gn(x8)

def summ(t):
    return float(t.mean()), float(t.std(unbiased=False))

print("BN  batch=1 mean/std:", summ(y_bn_1))
print("BN  batch=8 mean/std:", summ(y_bn_8))
print("GN  batch=1 mean/std:", summ(y_gn_1))
print("GN  batch=8 mean/std:", summ(y_gn_8))

BN  batch=1 mean/std: (-3.812601789832115e-09, 0.9999950528144836)
BN  batch=8 mean/std: (-5.587935447692871e-09, 0.9999949932098389)
GN  batch=1 mean/std: (-8.731149137020111e-11, 0.9999949932098389)
GN  batch=8 mean/std: (-2.342858351767063e-09, 0.9999949932098389)



**Yorum:**  
- BN çıktısı batch’e göre değişmeye daha yatkın (çünkü istatistiği batch’ten alıyor).  
- GN’de batch boyutu değişse de “normalize mantığı” sample içidir; daha stabil davranır.

> Bu demo “tam eğitim performansı” göstermez; sadece mekanizmayı sezdirir.



## 11) Pratik dönüşüm fonksiyonları (BN → GN)

Elimizde BN’li bir model varsa, BN’leri GN’e çevirmek için modül dolaşan bir fonksiyon kullanabiliriz.
Aşağıdaki fonksiyon `BatchNorm2d` gördüğü yerde onu `GroupNorm` ile değiştirir.


In [ ]:
import torch.nn as nn

def convert_bn_to_gn(model: nn.Module, num_groups: int = 32):
    for name, m in model.named_children():
        if isinstance(m, nn.BatchNorm2d):
            C = m.num_features
            g = min(num_groups, C)
            while C % g != 0 and g > 1:
                g -= 1
            setattr(model, name, nn.GroupNorm(g, C))
        else:
            convert_bn_to_gn(m, num_groups=num_groups)
    return model


## 12) Sonuç: Sağlıklı eğitim için “3 kilit”

1) **Norm seçimi:** GN (veya pretrained finetune ise Frozen BN)  
2) **Ölçek stabilizasyonu:** WS + GN (opsiyonel ama güçlü)  
3) **Optimizasyon emniyeti:** LR + warmup + AMP (+ clipping)

Bunları yaptığımızda “saçmalama”nın %80–90’ı gider.
Kalan kısım genelde dataset/augment/loss ayarıdır.;